### FASE 3 da Libertadores - Quartas de Final

In [106]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

### Dicionário com os IDs e Nomes dos times.

In [107]:
# Dicionário com ID -> Nome do time (gerado anteriormente)
nomes_por_id = {
    44574236: 'Analove10 ITAQUI GRANDE!!',
    13707047: 'Super Vasco f.c',

    18661583: 'pura bucha /botafogo',
    1273719: 'Texas Club 2025',

    3246608: 'Real SCI',
    50158297: 'Lá do Itaqui',

    49128587: 'TORRESMO COM PINGA',
    36359: 'KillerColorado',
    
}

In [145]:
# Lista com todos os dados que você passou
dados_torneio_quartas = [
    # Jogo 1
    ("Jogo 1", 44574236),    
    ("Jogo 1", 18661583), 

    # Jogo 2
    ("Jogo 2", 1273719),
    ("Jogo 2", 13707047),

    # Jogo 3
    ("Jogo 3", 3246608),
    ("Jogo 3", 36359),

    # Jogo 4
    ("Jogo 4", 49128587),
    ("Jogo 4", 50158297),
]

# Criar DataFrame base
df_torneio_quartas = pd.DataFrame(dados_torneio_quartas, columns=["Jogo", "ID do Time"])

# Adicionar Nome do Time usando o dicionário
df_torneio_quartas["Nome do Time"] = df_torneio_quartas["ID do Time"].map(nomes_por_id)

# Adicionar ID no Grupo
df_torneio_quartas["ID no Grupo"] = df_torneio_quartas.groupby("Jogo").cumcount() + 1
df_torneio_quartas["ID no Grupo"] = df_torneio_quartas["ID no Grupo"].astype(str) + "_" + df_torneio_quartas["Jogo"].str[-1]

# Reorganizar colunas
df_torneio_quartas_liberta = df_torneio_quartas[["Jogo", "ID do Time", "Nome do Time", "ID no Grupo"]]

df_liberta_jogo_1 = df_torneio_quartas_liberta[df_torneio_quartas_liberta["Jogo"] == "Jogo 1"]
df_liberta_jogo_2 = df_torneio_quartas_liberta[df_torneio_quartas_liberta["Jogo"] == "Jogo 2"]
df_liberta_jogo_3 = df_torneio_quartas_liberta[df_torneio_quartas_liberta["Jogo"] == "Jogo 3"]
df_liberta_jogo_4 = df_torneio_quartas_liberta[df_torneio_quartas_liberta["Jogo"] == "Jogo 4"]

# Lista de grupos
grupos = {
    "Jogo 1": df_liberta_jogo_1,
    "Jogo 2": df_liberta_jogo_2,
    "Jogo 3": df_liberta_jogo_3,
    "Jogo 4": df_liberta_jogo_4,
}

display(df_liberta_jogo_1)

,Jogo,ID do Time,Nome do Time,ID no Grupo
0,Jogo 1,44574236,Analove10 ITAQUI GRANDE!!,1_1
1,Jogo 1,18661583,pura bucha /botafogo,2_1


### Definição dos Confrontos das 2 Rodadas da Fase 3 da Libertadores (Rodada 13 A 14)

In [ ]:
# Rodada 1 - Fase 3 Libertadores (Equivalente a 13º Rodada do Campeonato Brasileiro) 
confrontos_1a_rodada = [
    # Jogo 1
    ("Jogo 1", "1_1", "2_1"),    

    # Jogo 2
    ("Jogo 2", "1_2", "2_2"),    

    # Jogo 3
    ("Jogo 3", "1_3", "2_3"),   

    # Jogo 4
    ("Jogo 4", "1_4", "2_4")  
]

# Rodada 2 - Fase 3 Libertadores (Equivalente a 14º Rodada do Campeonato Brasileiro)
confrontos_2a_rodada = [
    # Jogo 1
    ("Jogo 1", "2_1", "1_1"),

    # Jogo 2
    ("Jogo 2", "2_2", "1_2"),

    # Jogo 3
    ("Jogo 3", "2_3", "1_3"), 

    # Jogo 4
    ("Jogo 4", "2_4", "1_4") 
]

In [167]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])

# Junta com df_torneio para buscar dados dos mandantes
df_mandantes = df_torneio_quartas_liberta.rename(columns={
    "ID no Grupo": "Mandante_ID",
    "Nome do Time": "Mandante_Nome",
    "ID do Time": "Mandante_ID_Time"
})[["Jogo", "Mandante_ID", "Mandante_Nome", "Mandante_ID_Time"]]

# Junta com df_torneio para buscar dados dos visitantes
df_visitantes = df_torneio_quartas_liberta.rename(columns={
    "ID no Grupo": "Visitante_ID",
    "Nome do Time": "Visitante_Nome",
    "ID do Time": "Visitante_ID_Time"    
})[["Jogo", "Visitante_ID", "Visitante_Nome", "Visitante_ID_Time"]]

display(df_confrontos)

,Grupo,Mandante_ID,Visitante_ID
0,Jogo 1,1_1,2_1
1,Jogo 2,1_2,2_2
2,Jogo 3,1_3,2_3
3,Jogo 4,1_4,2_4


In [168]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])
df_confrontos["Rodada"] = 1  # <<< ADICIONAR ESTA LINHA
df_rodada_13 = df_confrontos.merge(df_mandantes, on=["Jogo", "Mandante_ID"])
df_rodada_13 = df_rodada_13.merge(df_visitantes, on=["Jogo", "Visitante_ID"])

# Transformar em DataFrame
df_confrontos_2 = pd.DataFrame(confrontos_2a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])
df_confrontos_2["Rodada"] = 2
df_rodada_14 = df_confrontos_2.merge(df_mandantes, on=["Jogo", "Mandante_ID"])
df_rodada_14 = df_rodada_14.merge(df_visitantes, on=["Jogo", "Visitante_ID"])

display(df_rodada_13)

,Jogo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Jogo 1,1_1,2_1,1,Analove10 ITAQUI GRANDE!!,44574236,pura bucha /botafogo,18661583
1,Jogo 2,1_2,2_2,1,Texas Club 2025,1273719,Super Vasco f.c,13707047
2,Jogo 3,1_3,2_3,1,Real SCI,3246608,KillerColorado,36359
3,Jogo 4,1_4,2_4,1,TORRESMO COM PINGA,49128587,Lá do Itaqui,50158297


In [169]:
df_rodadas = pd.concat([
    df_rodada_13,
    df_rodada_14
], ignore_index=True)

# Ajustar a numeração da rodada para refletir as rodadas do Cartola (Rodada 13 até 14)
df_rodadas["Rodada"] = df_rodadas["Rodada"] + 12  # Ex: 1 -> 13, 2 -> 14

df_rodadas.to_excel("confrontos_fase_3_libertadores.xlsx", index=False)

# Exibir os confrontos da fase 3 (Quartas de Final)
display(df_rodadas.head(8)) 

,Jogo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Jogo 1,1_1,2_1,13,Analove10 ITAQUI GRANDE!!,44574236,pura bucha /botafogo,18661583
1,Jogo 2,1_2,2_2,13,Texas Club 2025,1273719,Super Vasco f.c,13707047
2,Jogo 3,1_3,2_3,13,Real SCI,3246608,KillerColorado,36359
3,Jogo 4,1_4,2_4,13,TORRESMO COM PINGA,49128587,Lá do Itaqui,50158297
4,Jogo 1,2_1,1_1,14,pura bucha /botafogo,18661583,Analove10 ITAQUI GRANDE!!,44574236
5,Jogo 2,2_2,1_2,14,Super Vasco f.c,13707047,Texas Club 2025,1273719
6,Jogo 3,2_3,1_3,14,KillerColorado,36359,Real SCI,3246608
7,Jogo 4,2_4,1_4,14,Lá do Itaqui,50158297,TORRESMO COM PINGA,49128587


In [170]:
# Criar lista de dicionários no formato desejado
confrontos_js_fase_3 = []

for _, row in df_rodadas.iterrows():
    confronto = {
        "jogo": row["Jogo"],
        "rodada": int(row["Rodada"]),
        "mandante": {
            "id": int(row["Mandante_ID_Time"]),
            "nome": row["Mandante_Nome"]
        },
        "visitante": {
            "id": int(row["Visitante_ID_Time"]),
            "nome": row["Visitante_Nome"]        }
    }
    confrontos_js_fase_3.append(confronto)

# Converter para JSON formatado
json_str = json.dumps(confrontos_js_fase_3, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma variável global
with open("confrontos_fase3_libertadores.js", "w", encoding="utf-8") as f:
    f.write("const confrontosFase3 = ")
    f.write(json_str)
    f.write(";")

In [172]:
def exibir_confrontos(df_rodadas, rodada=None, jogo=None):
    """
    Filtra e exibe os confrontos por rodada e/ou jogo.
    
    Parâmetros:
    - df_rodadas: DataFrame com todos os confrontos
    - rodada: número da rodada (int ou None para todas)
    - jogo: nome do jogo (str ou None para todos)
    
    Retorna:
    - DataFrame filtrado com as colunas relevantes
    """
    colunas = ["Rodada", "Jogo", "Mandante_Nome", "Visitante_Nome"]
    df_filtrado = df_rodadas.copy()

    df_filtrado["Rodada"] = df_filtrado["Rodada"].astype(str) + "ª Rodada"    

    if rodada is not None:
        df_filtrado = df_filtrado[df_filtrado["Rodada"] == rodada]

    if jogo is not None:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    return df_filtrado[colunas].sort_values(by=["Jogo", "Rodada"])

In [ ]:
# Exibir todos os confrontos do Jogo 2
display(exibir_confrontos(df_rodadas, jogo="Jogo 2").head(8))

,Rodada,Jogo,Mandante_Nome,Visitante_Nome
1,13ª Rodada,Jogo 2,Texas Club 2025,Super Vasco f.c
5,14ª Rodada,Jogo 2,Super Vasco f.c,Texas Club 2025


In [174]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=13).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(13, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes


def gerar_df_pontuacoes(api, ids_times):
    rodada_atual = api.mercado().rodada_atual
    total_rodadas = 2

    if not campeonato_comecou(api, ids_times):
        print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(13, total_rodadas + 1)])
        df[:] = 0
    else:
        df = pd.DataFrame()
        for nome, time_id in ids_times.items():
            pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
            df[nome] = pd.Series(pontuacoes)
        df = df.transpose()
        df.columns = [f'Rodada {i}' for i in range(13, rodada_atual)]
        df.loc['Lider_Rodada'] = df.idxmax()
    
    return df

In [175]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)
display(df_pontuacoes.T)

,Analove10 ITAQUI GRANDE!!,Super Vasco f.c,pura bucha /botafogo,Texas Club 2025,Real SCI,Lá do Itaqui,TORRESMO COM PINGA,KillerColorado,Lider_Rodada
Rodada 13,118.74,93.51,106.21,111.91,103.81,120.25,111.21,111.11,Lá do Itaqui


In [177]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):

    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        jogo = confronto["Jogo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        # Ignorar confrontos ainda não disputados (com pontuação 0 ou ausente)
        if (
            pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante) or
            (pontos_mandante == 0 and pontos_visitante == 0)
        ):
            continue

        for time in [mandante, visitante]:
            if jogo not in estatisticas:
                estatisticas[jogo] = {}
            if time not in estatisticas[jogo]:
                estatisticas[jogo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0,
                    "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[jogo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[jogo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[jogo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[jogo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[jogo][mandante]["Pontos"] += 3
            estatisticas[jogo][mandante]["Vitórias"] += 1
            estatisticas[jogo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[jogo][visitante]["Pontos"] += 3
            estatisticas[jogo][visitante]["Vitórias"] += 1
            estatisticas[jogo][mandante]["Derrotas"] += 1
        else:
            estatisticas[jogo][mandante]["Pontos"] += 1
            estatisticas[jogo][visitante]["Pontos"] += 1
            estatisticas[jogo][mandante]["Empates"] += 1
            estatisticas[jogo][visitante]["Empates"] += 1

    # Gerar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Jogo": jogo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [
                stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        }) for jogo, times in estatisticas.items()
    ], ignore_index=True)

    # Ordenar e adicionar posição
    df_resultado = df_resultado.sort_values(
        by=["Jogo", "Pontos", "Vitórias", "Total Cartola", "Saldo Cartola"],
        ascending=[True, False, False, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Jogo").cumcount() + 1

    df_resultado_por_jogo = {
        jogo: df_resultado[df_resultado["Jogo"] == jogo] for jogo in df_resultado["Jogo"].unique()
    }

    return df_resultado, df_resultado_por_jogo


In [157]:
# # Conferir se df_rodadas está com colunas e valores corretos
# print(df_rodadas.dtypes)
# print(df_rodadas.head())

# # Conferir colunas do df_pontuacoes
# print(df_pontuacoes.columns.tolist())

# # Conferir se há confrontos válidos para a função
# for i, row in df_rodadas.iterrows():
#     rodada = row["Rodada"]
#     col = f"Rodada {rodada}"
#     if col not in df_pontuacoes.columns:
#         print(f"❌ Coluna {col} NÃO existe na pontuação.")
#     else:
#         print(f"✅ Coluna {col} OK.")

In [158]:
# # Leitura do arquivo da fase anterior (Fase 2)
# df_fase2 = pd.read_excel("classificacao_por_grupo_fase_2.xlsx", sheet_name=None)
# df_total_fase2 = pd.concat(df_fase2.values(), ignore_index=True)

# # Padroniza nomes para evitar erros de comparação
# # df_total_fase2["Nome do Time"] = df_total_fase2["Nome do Time"].str.strip().str.lower()

# # Cria o dicionário com os totais da Fase 2 (para usar como base da Fase 3)
# total_fase3_por_time = df_total_fase2.set_index("Nome do Time")["Total Cartola"].to_dict()

In [159]:
# display(total_fase3_por_time)

In [178]:
# Padroniza os nomes no df_rodadas
df_rodadas["Mandante_Nome"] = df_rodadas["Mandante_Nome"].str.strip()
df_rodadas["Visitante_Nome"] = df_rodadas["Visitante_Nome"].str.strip()

# Padroniza os índices do df_pontuacoes
df_pontuacoes.index = df_pontuacoes.index.str.strip()

# Exibe para conferência
display(df_pontuacoes)

,Rodada 13
Analove10 ITAQUI GRANDE!!,118.74
Super Vasco f.c,93.51
pura bucha /botafogo,106.21
Texas Club 2025,111.91
Real SCI,103.81
Lá do Itaqui,120.25
TORRESMO COM PINGA,111.21
KillerColorado,111.11
Lider_Rodada,Lá do Itaqui


In [179]:
# Gerar a classificação da fase 3
df_resultado, df_resultado_por_jogo = classificacao_por_grupo(
    df_rodadas, df_pontuacoes
)

# Salvar cada grupo em uma aba do Excel
with pd.ExcelWriter("classificacao_por_grupo_fase_3.xlsx") as writer:
    for jogo, df in df_resultado_por_jogo.items():
        df.to_excel(writer, sheet_name=jogo, index=False)

# Exibir a classificação geral
df_resultado_jogo_1 = df_resultado[df_resultado["Jogo"] == "Jogo 1"]
df_resultado_jogo_2 = df_resultado[df_resultado["Jogo"] == "Jogo 2"]
df_resultado_jogo_3 = df_resultado[df_resultado["Jogo"] == "Jogo 3"]
df_resultado_jogo_4 = df_resultado[df_resultado["Jogo"] == "Jogo 4"]

display(df_resultado_jogo_1, df_resultado_jogo_2, df_resultado_jogo_3, df_resultado_jogo_4)


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
0,Jogo 1,Analove10 ITAQUI GRANDE!!,3,1,0,0,118.74,106.21,12.53,1
1,Jogo 1,pura bucha /botafogo,0,0,0,1,106.21,118.74,-12.53,2


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
2,Jogo 2,Texas Club 2025,3,1,0,0,111.91,93.51,18.40,1
3,Jogo 2,Super Vasco f.c,0,0,0,1,93.51,111.91,-18.40,2


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
5,Jogo 3,KillerColorado,3,1,0,0,111.11,103.81,7.30,1
4,Jogo 3,Real SCI,0,0,0,1,103.81,111.11,-7.30,2


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
7,Jogo 4,Lá do Itaqui,3,1,0,0,120.25,111.21,9.04,1
6,Jogo 4,TORRESMO COM PINGA,0,0,0,1,111.21,120.25,-9.04,2


In [180]:
# Verifica se a variável df_resultado_por_grupo existe e está populada
if 'df_resultado_por_jogo' in locals() and df_resultado_por_jogo:
    # Criar estrutura em formato de dicionário para JSON/JS
    classificacao_js_fase_3 = {}

    for jogo, df in df_resultado_por_jogo.items():
        classificacao_js_fase_3[jogo] = []
        for _, row in df.iterrows():
            classificacao_js_fase_3[jogo].append({
                "posicao": int(row["Posição"]),
                "nome": row["Nome do Time"],
                "pontos": int(row["Pontos"]),
                "vitorias": int(row["Vitórias"]),
                "empates": int(row["Empates"]),
                "derrotas": int(row["Derrotas"]),
                "totalCartola": float(row["Total Cartola"]),
                "cartolaSofrido": float(row["Cartola Sofrido"]),
                "saldoCartola": float(row["Saldo Cartola"])
            })

    # Converter para JSON formatado
    json_str = json.dumps(classificacao_js_fase_3, indent=2, ensure_ascii=False)

    # Salvar como arquivo JS com uma variável global
    with open("classificacao_por_grupo_fase_3.js", "w", encoding="utf-8") as f:
        f.write("const classificacaoFase3 = ")
        f.write(json_str)
        f.write(";")

    print("✅ Arquivo JS salvo com sucesso.")

else:
    print("⚠️ Classificação da fase 3 indisponível. O mercado pode estar em manutenção ou os dados ainda não foram gerados.")



✅ Arquivo JS salvo com sucesso.


In [181]:
def exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada, jogo=None):
    """
    Exibe os resultados de uma rodada específica, com pontuação e dados dos times.
    """

    if rodada not in df_rodadas["Rodada"].values:
        return pd.DataFrame([{
            "Jogo": jogo or "-",
            "Rodada": rodada,
            "Mandante_Nome": "-",          
            "Mandante_Pontos": "-",
            "Visitante_Nome": "-",           
            "Visitante_Pontos": "-",
        }])

    df_filtrado = df_rodadas[df_rodadas["Rodada"] == rodada]
    if jogo:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    resultados = []

    for _, row in df_filtrado.iterrows():
        jogo_ = row["Jogo"]
        mandante = row["Mandante_Nome"]
        visitante = row["Visitante_Nome"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "Jogo": jogo_,
            "Rodada": rodada,
            "Mandante_Nome": mandante,
            "Mandante_Pontos": pontos_mandante,
            "Visitante_Nome": visitante,
            "Visitante_Pontos": pontos_visitante
        })

    return pd.DataFrame(resultados)


In [182]:
# Exibir resultados da 13ª rodada
df_resultados_rodada_13 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=13)

# Exibir apenas os resultados do Grupo B na 1ª rodada
df_resultados_jogo_1 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=13, jogo="Jogo 1")

# Exibir
display(df_resultados_rodada_13)

,Jogo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Jogo 1,13,Analove10 ITAQUI GRANDE!!,118.74,pura bucha /botafogo,106.21
1,Jogo 2,13,Texas Club 2025,111.91,Super Vasco f.c,93.51
2,Jogo 3,13,Real SCI,103.81,KillerColorado,111.11
3,Jogo 4,13,TORRESMO COM PINGA,111.21,Lá do Itaqui,120.25


In [184]:
# Criar arquivo com uma aba para cada rodada contendo os resultados detalhados
from pathlib import Path

# Caminho do arquivo de saída
caminho_resultados = "resultados_fase_3.xlsx"

# Descobrir as rodadas únicas no DataFrame
rodadas_disponiveis = sorted(df_rodadas["Rodada"].unique())

with pd.ExcelWriter(caminho_resultados) as writer:
    for rodada in rodadas_disponiveis:
        df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
        nome_aba = f"Rodada {rodada}"
        df_resultados.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"Arquivo salvo com sucesso: {Path(caminho_resultados).resolve()}")

display(df_resultados)

Arquivo salvo com sucesso: C:\Users\ferna\Projetos\GitHub\cartola_fbc\libertadores\datasets_liberta\resultados_fase_3.xlsx


,Jogo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Jogo 1,14,pura bucha /botafogo,None,Analove10 ITAQUI GRANDE!!,None
1,Jogo 2,14,Super Vasco f.c,None,Texas Club 2025,None
2,Jogo 3,14,KillerColorado,None,Real SCI,None
3,Jogo 4,14,Lá do Itaqui,None,TORRESMO COM PINGA,None


In [185]:
resultados_js = []

for rodada in sorted(df_rodadas["Rodada"].unique()):
    df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
    
    for _, row in df_resultados.iterrows():
        resultado = {
            "jogo": row["Jogo"],
            "rodada": int(rodada),
            "mandante": {
                "nome": row["Mandante_Nome"],
                "pontos": float(row["Mandante_Pontos"]) if row["Mandante_Pontos"] is not None else None
            },
            "visitante": {
                "nome": row["Visitante_Nome"],
                "pontos": float(row["Visitante_Pontos"]) if row["Visitante_Pontos"] is not None else None               
            },
            "vencedor": (
                "mandante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] > row["Visitante_Pontos"]
                else "visitante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] < row["Visitante_Pontos"]
                else "empate" if row["Mandante_Pontos"] == row["Visitante_Pontos"] and row["Mandante_Pontos"] is not None
                else "indefinido"
            )

        }
        resultados_js.append(resultado)


# Exportar para arquivo .js
import json

with open("resultados_fase_3.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase3 = ")
    f.write(json.dumps(resultados_js, indent=2, ensure_ascii=False))
    f.write(";")
